In [238]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


StatementMeta(Sparkdemopool, 3, 254, Finished, Available)

In [242]:
# Constants
Ingestion_base_path = "abfss://raw@datademosynapse.dfs.core.windows.net/Stream/"
event_hub='azurefglocaldemo'
consumer_group='$Default' 
key = mssparkutils.credentials.getSecret('glocalazure2023', 'eventhubconnection','AzureKeyVault1')

StatementMeta(Sparkdemopool, 3, 258, Finished, Available)

## Reading stream from eventhub with configuration defined

In [243]:
ehConf = {
'eventhubs.connectionString':sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(key), 
'ehName':event_hub, 
'eventhubs.consumerGroup':consumer_group 
} 

event_hub_stream_df = (
  spark
    .readStream
    .format("eventhubs")
    .options(**ehConf)
    .load()
)


StatementMeta(Sparkdemopool, 3, 259, Finished, Available)

## Casting the Body to string

In [244]:
event_hub_stream_cast = event_hub_stream_df.withColumn("body", event_hub_stream_df["body"].cast("string"))

StatementMeta(Sparkdemopool, 3, 260, Finished, Available)

## Ingesting stream data to a delta table 

In [245]:
Query = (event_hub_stream_cast.writeStream.format("delta").option("checkpointlocation",f"{Ingestion_base_path}/_checkpoint").trigger(processingTime='30 seconds').outputMode("append").start(Ingestion_base_path)).awaitTermination()
Query.processAllAvailable()
Query.stop()
     

StatementMeta(Sparkdemopool, 3, 261, Finished, Available)

In [246]:
#check_df = spark.read.format("delta").load(Ingestion_base_path)
#display(check_df )
     

StatementMeta(Sparkdemopool, 3, 262, Finished, Available)

SynapseWidget(Synapse.DataFrame, 87ab1172-abb6-49fa-a822-d50595395a63)